"""classical_models.ipynb
by: Archie Gertsman (arkadiy2@illinois.edu)
Project director: Richard Sowers
r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/
Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license
"""

In [1]:
import warnings; warnings.simplefilter('ignore')

In [84]:
import sys
sys.path.append('../../src/')
sys.path.append('../../data/')
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from time import time

### load data

In [85]:
df = pd.read_pickle('../../data/block4_updated.pkl')

agg_dict={
    'xtrack_dist': ['std'],
    'avg_surr_speed': ['mean','std'],
    'lanes':['mean'],
    'len':['mean'],
    'speed':['mean','std'],
    'vehicle_density': ['mean'],
    'lon_acc': ['mean','std'],
    'lat_acc': ['mean','std']
}

df = df[list(agg_dict.keys()) + ['type']]
df.index = [df.index.map(lambda idx: f'{idx[0]}_{idx[1]}'),  
            df.index.get_level_values(2),  
            df.index.get_level_values(3)]
df.index.names = ['id','road','time']
df

xtrack_dist  avg_surr_speed  lanes  \
id       road                 time                                         
4_1_1    250699362_250699984  42.00     -1.883401       10.464171    5.4   
                              42.04     -1.980795       10.457843    5.4   
                              42.08     -1.937041       10.452857    5.4   
                              42.12     -1.893287       10.448586    5.4   
                              42.16     -1.941984       10.444986    5.4   
...                                           ...             ...    ...   
1_3_2116 300400248_8446047162 921.64     0.658316       26.005850    6.5   
                              921.68     0.764478       26.008150    6.5   
                              921.72     0.736857       26.010500    6.5   
                              921.76     0.783032       26.012950    6.5   
                              921.80     0.755411       26.015600    6.5   

                                          len    speed  vehicle_density  \
id       road                 time                                        
4_1_1    250699362_250699984  42.00    97.581  11.9046                7   
                              42.04    97.581  11.8975                7   
                              42.08    97.581  11.8919                7   
                              42.12    97.581  11.8871                7   
                              42.16    97.581  11.8831                7   
...                                       ...      ...              ...   
1_3_2116 300400248_8446047162 921.64  106.837  25.3482                2   
                              921.68  106.837  25.3217                2   
                              921.72  106.837  25.2946                2   
                              921.76  106.837  25.2667                2   
                              921.80  106.837  25.2370                2   

                                      lon_acc  lat_acc  type  
id       road                 time                            
4_1_1    250699362_250699984  42.00   -0.1145   0.0138  Taxi  
                              42.04   -0.1007   0.0147  Taxi  
                              42.08   -0.0918   0.0157  Taxi  
                              42.12   -0.0869   0.0167  Taxi  
                              42.16   -0.0784   0.0176  Taxi  
...                                       ...      ...   ...  
1_3_2116 300400248_8446047162 921.64  -0.3011   0.3234   Car  
                              921.68  -0.3032   0.3216   Car  
                              921.72  -0.3012   0.3180   Car  
                              921.76  -0.3054   0.3124   Car  
                              921.80  -0.3135   0.3045   Car  

[2997687 rows x 9 columns]

In [86]:
feature_list = ['_'.join([key,val]) for key,vals in agg_dict.items() for val in vals]
feature_list

['xtrack_dist_std',
 'avg_surr_speed_mean',
 'avg_surr_speed_std',
 'lanes_mean',
 'len_mean',
 'speed_mean',
 'speed_std',
 'vehicle_density_mean',
 'lon_acc_mean',
 'lon_acc_std',
 'lat_acc_mean',
 'lat_acc_std']

### create df_agg

In [559]:
%%time

window = 60
overlap = 0.3

step = int((1-overlap)*window)

def f(grp):
    lst = [grp.iloc[i:i+window] for i in range(0, (len(grp)-window), step)]
    if len(lst)==0:
        return None
    a = np.array(lst)
    a = np.concatenate( (a.mean(axis=1), a.std(axis=1) ), axis=1 )
    a = pd.DataFrame(a)
    return a

df_agg = df.drop('type',axis=1).groupby(['id','road']).apply(f)

df_agg.columns = ['_'.join([col,agg]) for agg in ['mean','std'] for col in df.drop('type',axis=1).columns]
df_agg = df_agg[feature_list]

vehicle_types = df.type.groupby(['id','road']).first()
df_agg = df_agg.reset_index(-1, drop=True).join(vehicle_types)

df_agg

CPU times: user 13.7 s, sys: 402 ms, total: 14.1 s
Wall time: 14.1 s


xtrack_dist_std  avg_surr_speed_mean  \
id       road                                                        
1_2_1013 250691795_250699359         0.122811            35.059769   
         250691795_250699359         0.370038            35.278806   
         250691795_250699359         0.289755            38.755763   
         250691795_250699359         0.131347            37.777329   
         250691795_250699359         0.072858            34.481129   
...                                       ...                  ...   
4_3_994  250706958_388172075         0.363527            20.812448   
         250706958_388172075         0.068874            25.045568   
         250706958_388172075         0.069196            26.424028   
         250706958_388172075         0.088929            26.748915   
         250706958_388172075         0.241447            25.771680   

                              avg_surr_speed_std  lanes_mean  len_mean  \
id       road                                                            
1_2_1013 250691795_250699359            1.933372        13.0   171.188   
         250691795_250699359            2.120265        13.0   171.188   
         250691795_250699359            1.861978        13.0   171.188   
         250691795_250699359            2.013567        13.0   171.188   
         250691795_250699359            1.851485        13.0   171.188   
...                                          ...         ...       ...   
4_3_994  250706958_388172075            2.786976         6.0    87.080   
         250706958_388172075            1.526220         6.0    87.080   
         250706958_388172075            0.420449         6.0    87.080   
         250706958_388172075            0.167431         6.0    87.080   
         250706958_388172075            0.863210         6.0    87.080   

                              speed_mean  speed_std  vehicle_density_mean  \
id       road                                                               
1_2_1013 250691795_250699359   38.784775   2.168499              4.516667   
         250691795_250699359   41.595718   0.342866              4.550000   
         250691795_250699359   41.682442   0.374225              3.800000   
         250691795_250699359   39.320633   1.649412              3.850000   
         250691795_250699359   34.691568   2.136059              4.183333   
...                                  ...        ...                   ...   
4_3_994  250706958_388172075   20.406982   1.959130              1.066667   
         250706958_388172075   24.640102   1.353915              1.066667   
         250706958_388172075   26.424028   0.420449              1.000000   
         250706958_388172075   26.748915   0.167431              1.000000   
         250706958_388172075   25.771680   0.863210              1.000000   

                              lon_acc_mean  lon_acc_std  lat_acc_mean  \
id       road                                                           
1_2_1013 250691795_250699359      0.866777     0.324213      0.233228   
         250691795_250699359      0.166950     0.255385      0.111577   
         250691795_250699359     -0.139460     0.263612     -0.174677   
         250691795_250699359     -0.655498     0.125555     -0.048875   
         250691795_250699359     -0.851812     0.046481     -0.043438   
...                                    ...          ...           ...   
4_3_994  250706958_388172075      0.809315     0.064972      0.639498   
         250706958_388172075      0.506415     0.322240      0.163750   
         250706958_388172075      0.135133     0.093416     -0.041227   
         250706958_388172075     -0.032203     0.089135      0.102667   
         250706958_388172075     -0.304382     0.224075      0.131572   

                              lat_acc_std type  
id       road                                   
1_2_1013 250691795_250699359     0.038109  Car  
         250691795_250699359     0.166141  Car  
        

### create df_agg_test by selecting subset from df_agg

In [561]:
test_size = 150 # per each calss

df_reset = df_agg.reset_index(level=1)

df_list = []
for t in ['Car','Taxi']:
    idx = df_reset[df_reset.type==t] \
        .index \
        .unique() \
        .to_series() \
        .sample(test_size) \
        .values
    df_list.append(df_reset.loc[idx])

df_agg_test = pd.concat(df_list).set_index('road', append=True)
df_agg_test

xtrack_dist_std  avg_surr_speed_mean  \
id      road                                                         
1_2_462 250691795_250699359          0.053178            35.833174   
        250691795_250699359          0.089228            33.708068   
        250691795_250699359          0.077971            27.733702   
        250691795_250699359          0.051618            24.526604   
        250691795_250699359          0.094011            31.353086   
...                                       ...                  ...   
1_3_330 300400248_8446047162         0.127817            32.260453   
        300400248_8446047162         0.071094            27.848633   
        300400248_8446047162         0.114765            18.116644   
        300400248_8446047162         0.161240            25.003121   
        300400248_8446047162         0.183092            24.430197   

                              avg_surr_speed_std  lanes_mean  len_mean  \
id      road                                                             
1_2_462 250691795_250699359             1.074687        13.0   171.188   
        250691795_250699359             1.169239        13.0   171.188   
        250691795_250699359             4.179981        13.0   171.188   
        250691795_250699359             5.693338        13.0   171.188   
        250691795_250699359             2.270605        13.0   171.188   
...                                          ...         ...       ...   
1_3_330 300400248_8446047162            0.500061         6.5   106.837   
        300400248_8446047162            5.955711         6.5   106.837   
        300400248_8446047162            2.767224         6.5   106.837   
        300400248_8446047162            4.636457         6.5   106.837   
        300400248_8446047162            6.214623         6.5   106.837   

                              speed_mean  speed_std  vehicle_density_mean  \
id      road                                                                
1_2_462 250691795_250699359    33.424540   0.767888              2.933333   
        250691795_250699359    33.009427   1.072145              2.800000   
        250691795_250699359    30.230898   0.990995              3.616667   
        250691795_250699359    30.431833   1.108707              3.433333   
        250691795_250699359    32.129023   0.253184              2.616667   
...                                  ...        ...                   ...   
1_3_330 300400248_8446047162   32.054158   0.421163              1.216667   
        300400248_8446047162   31.114672   0.532445              1.916667   
        300400248_8446047162   29.322342   0.941931              2.650000   
        300400248_8446047162   28.163723   0.842997              1.900000   
        300400248_8446047162   30.789622   1.263875              1.766667   

                              lon_acc_mean  lon_acc_std  lat_acc_mean  \
id      road                                                            
1_2_462 250691795_250699359       0.312905     0.332946      0.132215   
        250691795_250699359      -0.365330     0.280452     -0.113300   
        250691795_250699359      -0.352573     0.255634      0.081545   
        250691795_250699359       0.329162     0.282647      0.037642   
        250691795_250699359       0.140122     0.190078     -0.030850   
...                                    ...          ...           ...   
1_3_330 300400248_8446047162     -0.167380     0.114070     -0.077410   
        300400248_8446047162     -0.195617     0.103671      0.029910   
        300400248_8446047162     -0.374880     0.067945      0.101270   
        300400248_8446047162      0.186353     0.563682      0.005418   
        300400248_8446047162      0.199678     0.819767      0.017505   

                              lat_acc_std  type  
id      road                                     
1_2_462 250691795_250699359      0.074716   Car  
        250691795_250699359      0.314307   Car  
    

In [562]:
df_agg_test.groupby(['id']).first().type.value_counts()

Taxi    150
Car     150
Name: type, dtype: int64

In [519]:
df.groupby(['id']).first().type.value_counts()

Car     1794
Taxi     756
Name: type, dtype: int64

### create df_agg_train by selecting df_agg - df_agg_test

In [564]:
df_agg_train = df_agg.drop(df_agg_test.index)
# df_agg_train

def balance_road(road):
    class_counts = road.groupby('id').first().type.value_counts()
    n_resample = class_counts.max() - class_counts.min()
    road.reset_index(1, inplace=True, drop=True)
    idx_resample = road[road.type==class_counts.idxmin()] \
        .index \
        .unique() \
        .to_series() \
        .sample(n_resample, replace=True) \
        .values
    resample = road.loc[idx_resample]
    return pd.concat([road,resample])

df_agg_train.groupby(['road']).apply(balance_road)

# class_counts = df_agg_train.groupby(['road']).first().type.value_counts()
# n_resample = class_counts['Car'] - class_counts['Taxi']

# df_agg_train.reset_index((0,2), inplace=True)

# idx_resample = df_agg_train[df_agg_train.type=='Taxi'] \
#     .index \
#     .unique() \
#     .to_series() \
#     .sample(n_resample, replace=True) \
#     .values

# resample = df_agg_train.loc[idx_resample]
# # resample.index = [ resample.index.map(lambda idx: f'{idx[0]}r'), resample.index.get_level_values(1) ]
# df_agg_train = pd.concat([df_agg_train, resample]).set_index(['id','level_2'], append=True)
# df_agg_train

xtrack_dist_std  avg_surr_speed_mean  \
road                 id                                               
250691795_250699359  1_2_1013         0.122811            35.059769   
                     1_2_1013         0.370038            35.278806   
                     1_2_1013         0.289755            38.755763   
                     1_2_1013         0.131347            37.777329   
                     1_2_1013         0.072858            34.481129   
...                                        ...                  ...   
300400248_8446047162 1_3_1695         0.099218            32.566155   
                     1_3_1695         0.124702            32.857952   
                     1_3_1695         0.033231            29.947548   
                     1_3_1695         0.099363            28.734585   
                     1_3_1695         0.063282            26.426444   

                               avg_surr_speed_std  lanes_mean  len_mean  \
road                 id                                                   
250691795_250699359  1_2_1013            1.933372        13.0   171.188   
                     1_2_1013            2.120265        13.0   171.188   
                     1_2_1013            1.861978        13.0   171.188   
                     1_2_1013            2.013567        13.0   171.188   
                     1_2_1013            1.851485        13.0   171.188   
...                                           ...         ...       ...   
300400248_8446047162 1_3_1695            0.672884         6.5   106.837   
                     1_3_1695            1.050751         6.5   106.837   
                     1_3_1695            2.189763         6.5   106.837   
                     1_3_1695            2.876138         6.5   106.837   
                     1_3_1695            1.661772         6.5   106.837   

                               speed_mean  speed_std  vehicle_density_mean  \
road                 id                                                      
250691795_250699359  1_2_1013   38.784775   2.168499              4.516667   
                     1_2_1013   41.595718   0.342866              4.550000   
                     1_2_1013   41.682442   0.374225              3.800000   
                     1_2_1013   39.320633   1.649412              3.850000   
                     1_2_1013   34.691568   2.136059              4.183333   
...                                   ...        ...                   ...   
300400248_8446047162 1_3_1695   32.788327   0.681491              1.366667   
                     1_3_1695   31.921142   1.504513              1.533333   
                     1_3_1695   28.655988   1.075555              1.683333   
                     1_3_1695   26.224325   0.936802              2.050000   
                     1_3_1695   23.719278   1.380572              2.216667   

                               lon_acc_mean  lon_acc_std  lat_acc_mean  \
road                 id                                                  
250691795_250699359  1_2_1013      0.866777     0.324213      0.233228   
                     1_2_1013      0.166950     0.255385      0.111577   
                     1_2_1013     -0.139460     0.263612     -0.174677   
                     1_2_1013     -0.655498     0.125555     -0.048875   
                     1_2_1013     -0.851812     0.046481     -0.043438   
...                                     ...          ...           ...   
300400248_8446047162 1_3_1695      0.209202     0.221018      0.104783   
                     1_3_1695     -0.475437     0.343092     -0.067662   
                     1_3_1695     -0.451003     0.078716      0.015100   
                     1_3_1695     -0.365123     0.052290      0.024827   
                     1_3_1695     -0.568797     0.279062     -0.021293   

                               lat_acc_std  type  
road                 id                           
250691795_250699359  1_2_1013     0.038109   Car  


In [543]:
class_counts

Car     13
Taxi     5
Name: type, dtype: int64

In [535]:
# df_agg_train.groupby(['id','road']).first().type.value_counts()

### define accuracy score by two levels of voting

In [536]:
def accuracy(y, y_hat):
    y_hat = y_hat.groupby(['id','road']).agg(lambda x: x.mode()[0])
    y_hat = y_hat.groupby('id').agg(lambda x: x.mode()[0])
#     y_hat = y_hat.groupby(['id','road']).agg('mean')
#     y_hat = y_hat.groupby('id').agg('mean')
#     y_hat = y_hat.type.map(lambda x: 'Car' if x>=0.5 else 'Taxi')

    y = y.groupby('id').first()

    return accuracy_score(y, y_hat)

### train

In [537]:
# baseline
# n_cars, n_taxis = df_agg_train.groupby('id').first().type.value_counts()
# n_cars / (n_cars+n_taxis)

In [565]:
model = GradientBoostingClassifier()
X,y = df_agg_train.drop('type', axis=1), df_agg_train.type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True)

model.fit(X_train,y_train)

y_hat = model.predict(X_test)
y_hat = pd.DataFrame(index=X_test.index, data=y_hat, columns=['type'])

accuracy(y_test, y_hat)

0.6904441453566622

### test

In [566]:
X,y = df_agg_test.drop('type', axis=1), df_agg_test.type

y_hat = model.predict(X)
y_hat = pd.DataFrame(index=X.index, data=y_hat, columns=['type'])

accuracy(y, y_hat)

0.5